# 1. Creación de Escenarios y Particiones por Consulta Clínica para cada día del Set de Pruebas.

## 1.1 Importación de Librerías y Carga del Set de Pruebas heredado de la Fase 1 - Predicción de No Shows.

In [1]:
import pandas as pd
from datetime import datetime
import random

from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl.utils.dataframe import dataframe_to_rows

In [2]:
# Ruta del archivo CSV con los datos del Set de Pruebas
file_path = "./test_set_p1-ALL full.csv"

# Leer el archivo CSV
test_set_full = pd.read_csv(file_path)

# Nos quedamos con el contenido que necesitamos: 'PatientId', 'ScheduledDay', 'AppointmentDay', 'NoShow'
test_set = test_set_full[['PatientId', 'ScheduledDay', 'AppointmentDay', 'NoShow']]

# Convertimos 'ScheduledDay' y 'AppointmentDay' a datetime
pd.options.mode.copy_on_write = True
test_set['ScheduledDay'] = pd.to_datetime(test_set['ScheduledDay'])
test_set['AppointmentDay'] = pd.to_datetime(test_set['AppointmentDay'])

# Mostrar el contenido del DataFrame
test_set

,PatientId,ScheduledDay,AppointmentDay,NoShow
0,39217,2016-05-31 10:56:41,2016-06-03,False
1,43741,2016-06-01 14:22:58,2016-06-01,False
2,22638656,2016-05-18 13:37:12,2016-06-08,False
3,61433777,2016-06-02 08:10:23,2016-06-02,False
4,79228495,2016-05-30 08:51:07,2016-06-08,False
...,...,...,...,...
22092,999277798221655,2016-06-07 15:39:48,2016-06-07,False
22093,999471678632216,2016-06-01 17:12:39,2016-06-06,False
22094,999743668137917,2016-06-08 12:55:37,2016-06-08,False
22095,999877415842165,2016-06-06 12:56:18,2016-06-06,False


In [3]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22097 entries, 0 to 22096
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   PatientId       22097 non-null  int64         
 1   ScheduledDay    22097 non-null  datetime64[ns]
 2   AppointmentDay  22097 non-null  datetime64[ns]
 3   NoShow          22097 non-null  bool          
dtypes: bool(1), datetime64[ns](2), int64(1)
memory usage: 539.6 KB


## 1.2 Creación de Set de Entrenamiento y Set de Validación.

Reservamos un 20% de los datos para verificar la Optimización en la Programación de Citas Médicas una vez entrenado el modelo.

In [4]:
# Calculate the total number of NoShows
total_no_shows = test_set['NoShow'].sum()

# Calculate the number of NoShows for each dataset
no_shows_80_percent = int(total_no_shows * 0.8)
no_shows_20_percent = total_no_shows - no_shows_80_percent

# Split the dataset into two based on the calculated proportions
train_set = test_set[test_set['NoShow'] == 0].sample(frac = 0.8, random_state = 42)
train_set = pd.concat([
        train_set,
        test_set[test_set['NoShow'] == 1].sample(n = no_shows_80_percent, random_state = 42)
])

val_set = test_set.drop(train_set.index)

# Verify the proportions of NoShows in each dataset
proportion_NoShows_train_set = train_set['NoShow'].mean()
proportion_NoShows_val_set = val_set['NoShow'].mean()

print(f'Proporción de NoShows en el Set de Entrenamiento: {proportion_NoShows_train_set:.4f}')
print(f'Proporción de NoShows en el Set de Validación: {proportion_NoShows_val_set:.4f}')

Proporción de NoShows en el Set de Entrenamiento: 0.2018
Proporción de NoShows en el Set de Validación: 0.2020


## 1.3 Separación de datos por 'AppointmentDay'.

Agrupamos los datos por 'AppointmentDay', revisamos y guardamos dichos subconjuntos. Aunque esto sólo será de aplicación cuando entrenemos el algoritmo de ML que calcule los mejores spots para generar el overbooking.

In [5]:
# Group the DataFrame by 'AppointmentDay'
grouped_train_data = train_set.groupby('AppointmentDay')
grouped_val_data = val_set.groupby('AppointmentDay')

# Create a dictionary to store the smaller datasets
train_datasets = {}
val_datasets = {}

# Iterate over each group and store the corresponding dataset in the dictionary
for day, group in grouped_train_data:
    train_datasets[day] = group
for day, group in grouped_val_data:
    val_datasets[day] = group

# Access, review and store the datasets for specific days
train_set_May30 = train_datasets[datetime(2016, 5, 30)]
print(f"El set de entrenamiento para el 30 de mayo de 2016 tiene \
{train_set_May30.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_May30['NoShow'].mean():.2%}.")
val_set_May30 = val_datasets[datetime(2016, 5, 30)]
print(f"El set de validación para el 30 de mayo de 2016 tiene \
{val_set_May30.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_May30['NoShow'].mean():.2%}.")
print("\033[1mDescartamos los datos del 30 de mayo de 2016 por \
insuficiencia y por ser todos No Show.\033[0m\n")

train_set_May31 = train_datasets[datetime(2016, 5, 31)]
print(f"El set de entrenamiento para el 31 de mayo de 2016 tiene \
{train_set_May31.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_May31['NoShow'].mean():.2%}.")
val_set_May31 = val_datasets[datetime(2016, 5, 31)]
print(f"El set de validación para el 31 de mayo de 2016 tiene \
{val_set_May31.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_May31['NoShow'].mean():.2%}.")
print("\033[1mDescartamos los datos del 31 de mayo de 2016 porque \
el alto número de No Shows desvirtuan la programación y cálculo de los costes.\033[0m\n")

train_set_June01 = train_datasets[datetime(2016, 6, 1)]
print(f"El set de entrenamiento para el 1 de junio de 2016 tiene \
{train_set_June01.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June01['NoShow'].mean():.2%}.")
val_set_June01 = val_datasets[datetime(2016, 6, 1)]
print(f"El set de validación para el 1 de junio de 2016 tiene \
{val_set_June01.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June01['NoShow'].mean():.2%}.\n")

train_set_June02 = train_datasets[datetime(2016, 6, 2)]
print(f"El set de entrenamiento para el 2 de junio de 2016 tiene \
{train_set_June02.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June02['NoShow'].mean():.2%}.")
val_set_June02 = val_datasets[datetime(2016, 6, 2)]
print(f"El set de validación para el 2 de junio de 2016 tiene \
{val_set_June02.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June02['NoShow'].mean():.2%}.\n")

train_set_June03 = train_datasets[datetime(2016, 6, 3)]
print(f"El set de entrenamiento para el 3 de junio de 2016 tiene \
{train_set_June03.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June03['NoShow'].mean():.2%}.")
val_set_June03 = val_datasets[datetime(2016, 6, 3)]
print(f"El set de validación para el 3 de junio de 2016 tiene \
{val_set_June03.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June03['NoShow'].mean():.2%}.\n")

train_set_June06 = train_datasets[datetime(2016, 6, 6)]
print(f"El set de entrenamiento para el 6 de junio de 2016 tiene \
{train_set_June06.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June06['NoShow'].mean():.2%}.")
val_set_June06 = val_datasets[datetime(2016, 6, 6)]
print(f"El set de validación para el 6 de junio de 2016 tiene \
{val_set_June06.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June06['NoShow'].mean():.2%}.\n")

train_set_June07 = train_datasets[datetime(2016, 6, 7)]
print(f"El set de entrenamiento para el 7 de junio de 2016 tiene \
{train_set_June07.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June07['NoShow'].mean():.2%}.")
val_set_June07 = val_datasets[datetime(2016, 6, 7)]
print(f"El set de validación para el 7 de junio de 2016 tiene \
{val_set_June07.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June07['NoShow'].mean():.2%}.\n")

train_set_June08 = train_datasets[datetime(2016, 6, 8)]
print(f"El set de entrenamiento para el 8 de junio de 2016 tiene \
{train_set_June08.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June08['NoShow'].mean():.2%}.")
val_set_June08 = val_datasets[datetime(2016, 6, 8)]
print(f"El set de validación para el 8 de junio de 2016 tiene \
{val_set_June08.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June08['NoShow'].mean():.2%}.\n")

print("\033[1mREHACEMOS TODO EL PROCESO DESCARTANDO LOS DATOS DEL 30 Y 31 DE MAYO.\033[0m\n")

El set de entrenamiento para el 30 de mayo de 2016 tiene 51 citas, con un porcentaje de NoShows del 100.00%.
El set de validación para el 30 de mayo de 2016 tiene 9 citas, con un porcentaje de NoShows del 100.00%.
Descartamos los datos del 30 de mayo de 2016 por insuficiencia y por ser todos No Show.

El set de entrenamiento para el 31 de mayo de 2016 tiene 616 citas, con un porcentaje de NoShows del 68.02%.
El set de validación para el 31 de mayo de 2016 tiene 149 citas, con un porcentaje de NoShows del 61.74%.
Descartamos los datos del 31 de mayo de 2016 porque el alto número de No Shows desvirtuan la programación y cálculo de los costes.

El set de entrenamiento para el 1 de junio de 2016 tiene 2482 citas, con un porcentaje de NoShows del 18.09%.
El set de validación para el 1 de junio de 2016 tiene 579 citas, con un porcentaje de NoShows del 21.59%.

El set de entrenamiento para el 2 de junio de 2016 tiene 2601 citas, con un porcentaje de NoShows del 18.38%.
El set de validación pa

In [6]:
# Quitamos las citas del 30 y 31 de mayo.
new_test_set = test_set[~test_set['AppointmentDay'].dt.strftime('%Y-%m-%d').isin(['2016-05-30', '2016-05-31'])]

print(new_test_set[new_test_set['AppointmentDay'] == datetime(2016, 5, 30)])
print(new_test_set[new_test_set['AppointmentDay'] == datetime(2016, 5, 31)])
new_test_set

Empty DataFrame
Columns: [PatientId, ScheduledDay, AppointmentDay, NoShow]
Index: []
Empty DataFrame
Columns: [PatientId, ScheduledDay, AppointmentDay, NoShow]
Index: []


,PatientId,ScheduledDay,AppointmentDay,NoShow
0,39217,2016-05-31 10:56:41,2016-06-03,False
1,43741,2016-06-01 14:22:58,2016-06-01,False
2,22638656,2016-05-18 13:37:12,2016-06-08,False
3,61433777,2016-06-02 08:10:23,2016-06-02,False
4,79228495,2016-05-30 08:51:07,2016-06-08,False
...,...,...,...,...
22092,999277798221655,2016-06-07 15:39:48,2016-06-07,False
22093,999471678632216,2016-06-01 17:12:39,2016-06-06,False
22094,999743668137917,2016-06-08 12:55:37,2016-06-08,False
22095,999877415842165,2016-06-06 12:56:18,2016-06-06,False


In [7]:
new_test_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21272 entries, 0 to 22096
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   PatientId       21272 non-null  int64         
 1   ScheduledDay    21272 non-null  datetime64[ns]
 2   AppointmentDay  21272 non-null  datetime64[ns]
 3   NoShow          21272 non-null  bool          
dtypes: bool(1), datetime64[ns](2), int64(1)
memory usage: 685.5 KB


In [8]:
# Calculate the total number of NoShows
total_no_shows = new_test_set['NoShow'].sum()

# Calculate the number of NoShows for each dataset
no_shows_80_percent = int(total_no_shows * 0.8)
no_shows_20_percent = total_no_shows - no_shows_80_percent

# Split the dataset into two based on the calculated proportions
train_set = new_test_set[new_test_set['NoShow'] == 0].sample(frac = 0.8, random_state = 42)
train_set = pd.concat([train_set,
        new_test_set[new_test_set['NoShow'] == 1].sample(n = no_shows_80_percent, random_state = 42)])

val_set = new_test_set.drop(train_set.index)

# Verify the proportions of NoShows in each dataset
proportion_NoShows_train_set = train_set['NoShow'].mean()
proportion_NoShows_val_set = val_set['NoShow'].mean()

print(f'Proporción de NoShows en el Set de Entrenamiento: {proportion_NoShows_train_set:.4f}')
print(f'Proporción de NoShows en el Set de Validación: {proportion_NoShows_val_set:.4f}')

Proporción de NoShows en el Set de Entrenamiento: 0.1829
Proporción de NoShows en el Set de Validación: 0.1829


In [9]:
# Group the DataFrame by 'AppointmentDay'
grouped_total_data = new_test_set.groupby('AppointmentDay')
grouped_train_data = train_set.groupby('AppointmentDay')
grouped_val_data = val_set.groupby('AppointmentDay')

# Create a dictionary to store the smaller datasets
total_datasets = {}
train_datasets = {}
val_datasets = {}

# Iterate over each group and store the corresponding dataset in the dictionary
for day, group in grouped_total_data:
    total_datasets[day] = group
for day, group in grouped_train_data:
    train_datasets[day] = group
for day, group in grouped_val_data:
    val_datasets[day] = group

# Access, review and store the datasets for specific days
total_set_June01 = total_datasets[datetime(2016, 6, 1)]
print(f"El set con todas las citas médicas para el 1 de junio de 2016 tiene \
{total_set_June01.shape[0]} citas, con un porcentaje de NoShows del \
{total_set_June01['NoShow'].mean():.2%}.")
train_set_June01 = train_datasets[datetime(2016, 6, 1)]
print(f"El set de entrenamiento para el 1 de junio de 2016 tiene \
{train_set_June01.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June01['NoShow'].mean():.2%}.")
val_set_June01 = val_datasets[datetime(2016, 6, 1)]
print(f"El set de validación para el 1 de junio de 2016 tiene \
{val_set_June01.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June01['NoShow'].mean():.2%}.\n")

total_set_June02 = total_datasets[datetime(2016, 6, 2)]
print(f"El set con todas las citas médicas para el 2 de junio de 2016 tiene \
{total_set_June02.shape[0]} citas, con un porcentaje de NoShows del \
{total_set_June02['NoShow'].mean():.2%}.")
train_set_June02 = train_datasets[datetime(2016, 6, 2)]
print(f"El set de entrenamiento para el 2 de junio de 2016 tiene \
{train_set_June02.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June02['NoShow'].mean():.2%}.")
val_set_June02 = val_datasets[datetime(2016, 6, 2)]
print(f"El set de validación para el 2 de junio de 2016 tiene \
{val_set_June02.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June02['NoShow'].mean():.2%}.\n")

total_set_June03 = total_datasets[datetime(2016, 6, 3)]
print(f"El set con todas las citas médicas para el 3 de junio de 2016 tiene \
{total_set_June03.shape[0]} citas, con un porcentaje de NoShows del \
{total_set_June03['NoShow'].mean():.2%}.")
train_set_June03 = train_datasets[datetime(2016, 6, 3)]
print(f"El set de entrenamiento para el 3 de junio de 2016 tiene \
{train_set_June03.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June03['NoShow'].mean():.2%}.")
val_set_June03 = val_datasets[datetime(2016, 6, 3)]
print(f"El set de validación para el 3 de junio de 2016 tiene \
{val_set_June03.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June03['NoShow'].mean():.2%}.\n")

total_set_June06 = total_datasets[datetime(2016, 6, 6)]
print(f"El set con todas las citas médicas para el 6 de junio de 2016 tiene \
{total_set_June06.shape[0]} citas, con un porcentaje de NoShows del \
{total_set_June06['NoShow'].mean():.2%}.")
train_set_June06 = train_datasets[datetime(2016, 6, 6)]
print(f"El set de entrenamiento para el 6 de junio de 2016 tiene \
{train_set_June06.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June06['NoShow'].mean():.2%}.")
val_set_June06 = val_datasets[datetime(2016, 6, 6)]
print(f"El set de validación para el 6 de junio de 2016 tiene \
{val_set_June06.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June06['NoShow'].mean():.2%}.\n")

total_set_June07 = total_datasets[datetime(2016, 6, 7)]
print(f"El set con todas las citas médicas para el 7 de junio de 2016 tiene \
{total_set_June07.shape[0]} citas, con un porcentaje de NoShows del \
{total_set_June07['NoShow'].mean():.2%}.")
train_set_June07 = train_datasets[datetime(2016, 6, 7)]
print(f"El set de entrenamiento para el 7 de junio de 2016 tiene \
{train_set_June07.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June07['NoShow'].mean():.2%}.")
val_set_June07 = val_datasets[datetime(2016, 6, 7)]
print(f"El set de validación para el 7 de junio de 2016 tiene \
{val_set_June07.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June07['NoShow'].mean():.2%}.\n")

total_set_June08 = total_datasets[datetime(2016, 6, 8)]
print(f"El set con todas las citas médicas para el 8 de junio de 2016 tiene \
{total_set_June08.shape[0]} citas, con un porcentaje de NoShows del \
{total_set_June08['NoShow'].mean():.2%}.")
train_set_June08 = train_datasets[datetime(2016, 6, 8)]
print(f"El set de entrenamiento para el 8 de junio de 2016 tiene \
{train_set_June08.shape[0]} citas, con un porcentaje de NoShows del \
{train_set_June08['NoShow'].mean():.2%}.")
val_set_June08 = val_datasets[datetime(2016, 6, 8)]
print(f"El set de validación para el 8 de junio de 2016 tiene \
{val_set_June08.shape[0]} citas, con un porcentaje de NoShows del \
{val_set_June08['NoShow'].mean():.2%}.\n")

El set con todas las citas médicas para el 1 de junio de 2016 tiene 3061 citas, con un porcentaje de NoShows del 18.75%.
El set de entrenamiento para el 1 de junio de 2016 tiene 2444 citas, con un porcentaje de NoShows del 18.82%.
El set de validación para el 1 de junio de 2016 tiene 617 citas, con un porcentaje de NoShows del 18.48%.

El set con todas las citas médicas para el 2 de junio de 2016 tiene 3248 citas, con un porcentaje de NoShows del 18.29%.
El set de entrenamiento para el 2 de junio de 2016 tiene 2597 citas, con un porcentaje de NoShows del 18.33%.
El set de validación para el 2 de junio de 2016 tiene 651 citas, con un porcentaje de NoShows del 18.13%.

El set con todas las citas médicas para el 3 de junio de 2016 tiene 3163 citas, con un porcentaje de NoShows del 19.60%.
El set de entrenamiento para el 3 de junio de 2016 tiene 2517 citas, con un porcentaje de NoShows del 19.86%.
El set de validación para el 3 de junio de 2016 tiene 646 citas, con un porcentaje de NoShows

## 1.4 Creación de grupos para aplicar Reglas de Asignación de Cita.

Para cada día de 'AppointmentDay', mezclamos aleatoriamente los datos, dividimos en grupos de 24 o 30 citas (para calcular los costes utilizando cada una de las dos Reglas de Asignación de cita definidas en nuestras hipótesis), descartamos el grupo que no alcance las 24 o 30 citas, y ordenamos según 'ScheduledDay'. De esta forma ya estarán listos los grupos para hacer asignación de cita de acuerdo a la Regla de Asignación definida y poder calcular los costes según la asistencia de los pacientes.

**Repetimos la operación S veces para S escenarios distintos de solicitud de cita, y así tener unos datos promedio de los costes más exactos.**

In [10]:
def create_groups(set: pd.DataFrame, N: int, S: int) -> dict:
    """
    Create groups of patients from a dataset.
    Args:
        set (dataset): The dataset containing patient information.
        N (int): The number of patients in each group.
        S (int): The number of different settings to take into account.
    Returns:
        dict: A dictionary containing the groups of patients for each random setting.
    """
    # Initiate the dictionary to store the groups
    groups = {}

    for setting in range(1, S+1):
        # Shuffle the set
        rs = setting
        shuffled_df = set.sample(frac = 1, random_state = rs)

        # Divide the shuffled DataFrame into groups of N number of patients
        # and discard the last one if it has less than this number N
        setting_name = f"setting {setting}"
        groups[setting_name] = [shuffled_df.iloc[i:i+N] for i
                                            in range(0, len(shuffled_df), N)]
        if len(groups[setting_name][-1]) < N:
            groups[setting_name] = groups[setting_name][:-1]

        # Order each group by 'ScheduledDay'
        groups[setting_name] = [group.sort_values('ScheduledDay')
                                for group in groups[setting_name]]

    return groups

### 1.4.1 Regla de Asignación de Cita: **1 Slot : 1 Patient**

En cada slot de 20min sólo se cita a un paciente.

Jornada laboral de 8 horas = 24 slots => 24 pacientes citados = N

In [11]:
# Grupos del 1 de junio con 10 settings distintos para 24 pacientes citados
total_groups_June01_R11 = create_groups(total_set_June01, N = 24 , S = 10)

# Grupos del 2 de junio con 10 settings distintos para 24 pacientes citados
total_groups_June02_R11 = create_groups(total_set_June02, N = 24 , S = 10)

# Grupos del 3 de junio con 10 settings distintos para 24 pacientes citados
total_groups_June03_R11 = create_groups(total_set_June03, N = 24 , S = 10)

# Grupos del 6 de junio con 10 settings distintos para 24 pacientes citados
total_groups_June06_R11 = create_groups(total_set_June06, N = 24 , S = 10)

# Grupos del 7 de junio con 10 settings distintos para 24 pacientes citados
total_groups_June07_R11 = create_groups(total_set_June07, N = 24 , S = 10)

# Grupos del 8 de junio con 10 settings distintos para 24 pacientes citados
total_groups_June08_R11 = create_groups(total_set_June08, N = 24 , S = 10)

### 1.4.2 Regla de Asignación de Cita: **1 Slot : 2 Patients - 3 Slots : 1 Patient**

En el primer slot de 20min se cita a dos pacientes. Luego se vuelve a citar un sólo paciente por slot durante 3 slots. Y se repite la secuencia.

Jornada laboral de 8 horas = 24 slots => 24 + 6 = 30 pacientes citados = N

In [12]:
# Grupos del 1 de junio con 10 settings distintos para 30 pacientes citados
total_groups_June01_R1231 = create_groups(total_set_June01, N = 30 , S = 10)

# Grupos del 2 de junio con 10 settings distintos para 30 pacientes citados
total_groups_June02_R1231 = create_groups(total_set_June02, N = 30 , S = 10)

# Grupos del 3 de junio con 10 settings distintos para 30 pacientes citados
total_groups_June03_R1231 = create_groups(total_set_June03, N = 30 , S = 10)

# Grupos del 6 de junio con 10 settings distintos para 30 pacientes citados
total_groups_June06_R1231 = create_groups(total_set_June06, N = 30 , S = 10)

# Grupos del 7 de junio con 10 settings distintos para 30 pacientes citados
total_groups_June07_R1231 = create_groups(total_set_June07, N = 30 , S = 10)

# Grupos del 8 de junio con 10 settings distintos para 30 pacientes citados
total_groups_June08_R1231 = create_groups(total_set_June08, N = 30 , S = 10)

# 2. Cálculo de Costes

## 2.1 Regla de Asignación de Cita
### **1 Slot : 1 Patient**

24 Slots - 24 pacientes citados

### 2.1.1 Asignaciones de cita

In [13]:
NUM_SLOTS_PER_DAY = 24

# 01 Junio 2016:
# 10 settings x 127 consultas = 1270 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June01_R11 = {}
for i, setting in enumerate(total_groups_June01_R11.keys()):
    for j, group in enumerate(total_groups_June01_R11[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June01_R11[group_name] = {}
        for slot in range(NUM_SLOTS_PER_DAY):
            slot_apps_total_June01_R11[group_name][slot+1] = [group.iloc[slot].loc['PatientId']]

# 02 Junio 2016:
# 10 settings x 135 consultas = 1350 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June02_R11 = {}
for i, setting in enumerate(total_groups_June02_R11.keys()):
    for j, group in enumerate(total_groups_June02_R11[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June02_R11[group_name] = {}
        for slot in range(NUM_SLOTS_PER_DAY):
            slot_apps_total_June02_R11[group_name][slot+1] = [group.iloc[slot].loc['PatientId']]

# 03 Junio 2016:
# 10 settings x 131 consultas = 1310 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June03_R11 = {}
for i, setting in enumerate(total_groups_June03_R11.keys()):
    for j, group in enumerate(total_groups_June03_R11[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June03_R11[group_name] = {}
        for slot in range(NUM_SLOTS_PER_DAY):
            slot_apps_total_June03_R11[group_name][slot+1] = [group.iloc[slot].loc['PatientId']]

# 06 Junio 2016:
# 10 settings x 158 consultas = 1580 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June06_R11 = {}
for i, setting in enumerate(total_groups_June06_R11.keys()):
    for j, group in enumerate(total_groups_June06_R11[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June06_R11[group_name] = {}
        for slot in range(NUM_SLOTS_PER_DAY):
            slot_apps_total_June06_R11[group_name][slot+1] = [group.iloc[slot].loc['PatientId']]

# 07 Junio 2016:
# 10 settings x 161 consultas = 1610 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June07_R11 = {}
for i, setting in enumerate(total_groups_June07_R11.keys()):
    for j, group in enumerate(total_groups_June07_R11[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June07_R11[group_name] = {}
        for slot in range(NUM_SLOTS_PER_DAY):
            slot_apps_total_June07_R11[group_name][slot+1] = [group.iloc[slot].loc['PatientId']]

# 08 Junio 2016:
# 10 settings x 171 consultas = 1710 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June08_R11 = {}
for i, setting in enumerate(total_groups_June08_R11.keys()):
    for j, group in enumerate(total_groups_June08_R11[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June08_R11[group_name] = {}
        for slot in range(NUM_SLOTS_PER_DAY):
            slot_apps_total_June08_R11[group_name][slot+1] = [group.iloc[slot].loc['PatientId']]

### 2.1.2 Cálculo de Costes: Idle Time, Waiting Time y Over Time

In [14]:
def calculo_costes(set:pd.DataFrame, groups: dict, slot_apps: dict):
    NUM_SETTINGS = len(groups)
    NUM_DOCTORS = len(groups['setting 1'])
    print(f'Número de consultas: {len(slot_apps)}  \
=  {NUM_SETTINGS} settings diferentes * {NUM_DOCTORS} doctores pasando consulta.')
    
    copied_slot_apps = slot_apps.copy()   # Hacemos una copia para no sobre escribir fuera

    idle_costs = {}   # Costos de inactividad del doctor
    waiting_costs = {}   # Costos de espera del paciente
    overtime_costs = {}   # Costos de tiempo extra del doctor

    for setting in range(1, NUM_SETTINGS+1):
        for consulta in range(1, NUM_DOCTORS+1):
            scenery = f'slot_apps_{setting}_{consulta}'
            # Parámetros iniciales
            idle_costs[scenery] = 24   # Coste máximo, no se ha pasado ninguna consulta.
            waiting_costs[scenery] = 0   # Coste mínimo, nadie ha esperado.
            overtime_costs[scenery] = 0   # Coste mínimo, sin trabajo extra.

            patients_waiting_at_13 = 0   # Pacientes esperando a medio día
            patients_waiting_at_18 = 0   # Pacientes esperanto por la tarde
            slot = 1  # Primer Slot
            while slot < 25:
                slot_ocupado = False   # Inicialmente
                idx_insertion = 0
                for patient in slot_apps[scenery][slot]:
                    patient_row_index = set[set['PatientId'] == patient].index
                    if slot_ocupado and set.loc[patient_row_index, 'NoShow'].any() == False:
                        if slot == 12:
                            patients_waiting_at_13 += 1
                            if patients_waiting_at_13 > 3:
                                print(f'¡OJO! El doctor no tiene el tiempo para \
atender a este paciente {patient} de 13:00 a 14:00.\nSe mueve al slot de las 14:00.')
                                waiting_costs[scenery] += patients_waiting_at_13 * 1
                                copied_slot_apps[scenery][slot+1].insert(idx_insertion, patient)
                                idx_insertion += 1
                            else:
                                overtime_costs[scenery] += 1
                                waiting_costs[scenery] += patients_waiting_at_13 * 1
                        elif slot == 24:
                            patients_waiting_at_18 += 1
                            overtime_costs[scenery] += 1
                            waiting_costs[scenery] += patients_waiting_at_18 * 1
                        else:
                            waiting_costs[scenery] += 1
                            copied_slot_apps[scenery][slot+1].insert(idx_insertion, patient)
                            idx_insertion += 1
                    elif set.loc[patient_row_index, 'NoShow'].any() == False:
                        slot_ocupado = True
                        idle_costs[scenery] -= 1
                slot += 1

    average_idle_cost = sum(list(idle_costs.values())) / len(slot_apps)
    print(f'La media de los Costes de Inactividad en los {NUM_SETTINGS} escenarios distintos \
para las consultas de los {NUM_DOCTORS} doctores distintos es de {average_idle_cost:.2f}.')

    average_waiting_cost = sum(list(waiting_costs.values())) / len(slot_apps)
    print(f'La media de los Costes de Espera de los Pacientes en los {NUM_SETTINGS} escenarios distintos \
para las consultas de los {NUM_DOCTORS} doctores distintos es de {average_waiting_cost:.2f}.')

    average_overtime_cost = sum(list(overtime_costs.values())) / len(slot_apps)
    print(f'La media de los Costes de Tiempo Extra en los {NUM_SETTINGS} escenarios distintos \
para las consultas de los {NUM_DOCTORS} doctores distintos es de {average_overtime_cost:.2f}.')
    
    costs = {'idle_costs': idle_costs,
            'waiting_costs': waiting_costs,
            'overtime_costs': overtime_costs}
    average_costs = {'average_idle_cost': average_idle_cost,
                    'average_waiting_cost': average_waiting_cost,
                    'average_overtime_cost': average_overtime_cost}
    
    return costs, average_costs

#### **Misma función con todos los prints, para entender el código y supervisar los cálculos cuando se ejecuta.**

_(Recomiendo correrlo sólo con un día para no bloquear el equipo)_

    def calculo_costes(set:pd.DataFrame, groups: dict, slot_apps: dict):
        NUM_SETTINGS = len(groups)
        NUM_DOCTORS = len(groups['setting 1'])
        print(f'Número de consultas: {len(slot_apps)}  \
    =  {NUM_SETTINGS} settings diferentes * {NUM_DOCTORS} doctores pasando consulta.')
        
        copied_slot_apps = slot_apps.copy()   # Hacemos una copia para no sobre escribir fuera
    
        idle_costs = {}   # Costos de inactividad del doctor
        waiting_costs = {}   # Costos de espera del paciente
        overtime_costs = {}   # Costos de tiempo extra del doctor
    
        for setting in range(1, NUM_SETTINGS+1):
            for consulta in range(1, NUM_DOCTORS+1):
                scenery = f'slot_apps_{setting}_{consulta}'
                print('\n', scenery)
                # Parámetros iniciales
                idle_costs[scenery] = 24   # Coste máximo, no se ha pasado ninguna consulta.
                print(f'Idle[{scenery}] -> {idle_costs[scenery]}')
                waiting_costs[scenery] = 0   # Coste mínimo, nadie ha esperado.
                print(f'Waiting[{scenery}] -> {waiting_costs[scenery]}')
                overtime_costs[scenery] = 0   # Coste mínimo, sin trabajo extra.
                print(f'Overtime[{scenery}] -> {overtime_costs[scenery]}')
    
                patients_waiting_at_13 = 0   # Pacientes esperando a medio día
                patients_waiting_at_18 = 0   # Pacientes esperanto por la tarde
                slot = 1  # Primer Slot
                print('EMPEZAMOS')
                while slot < 25:
                    slot_ocupado = False   # Inicialmente
                    print(f'Slot {slot}')
                    idx_insertion = 0
                    print(f'\tIdx_insertion = {idx_insertion}')
                    for patient in slot_apps[scenery][slot]:
                        print(f'\tPatient {patient} - Slot_ocupado = {slot_ocupado}')
                        patient_row_index = set[set['PatientId'] == patient].index
                        print(f"\t¿Faltó a la cita? {set.loc[patient_row_index, 'NoShow'].any()}")
                        if slot_ocupado and set.loc[patient_row_index, 'NoShow'].any() == False:
                            if slot == 12:
                                patients_waiting_at_13 += 1
                                print(f'\tPatients waiting at 13:00: {patients_waiting_at_13}')
                                if patients_waiting_at_13 > 3:
                                    print(f'¡OJO! El doctor no tiene el tiempo para \
    atender a este paciente {patient} de 13:00 a 14:00.\nSe mueve al slot de las 14:00.')
                                    waiting_costs[scenery] += patients_waiting_at_13 * 1
                                    copied_slot_apps[scenery][slot+1].insert(idx_insertion, patient)
                                    idx_insertion += 1
                                    print(f'\tIdx_insertion = {idx_insertion}')
                                else:
                                    overtime_costs[scenery] += 1
                                    print(f'\tOvertime_cost[{scenery}] -> {overtime_costs[scenery]}')
                                    waiting_costs[scenery] += patients_waiting_at_13 * 1
                                    print(f'\tWaiting_cost[{scenery}] -> {waiting_costs[scenery]}')
                            elif slot == 24:
                                patients_waiting_at_18 += 1
                                print(f'\tPatients waiting at 18:00: {patients_waiting_at_18}')
                                overtime_costs[scenery] += 1
                                print(f'\tOvertime_cost[{scenery}] -> {overtime_costs[scenery]}')
                                waiting_costs[scenery] += patients_waiting_at_18 * 1
                                print(f'\tWaiting_cost[{scenery}] -> {waiting_costs[scenery]}')
                            else:
                                waiting_costs[scenery] += 1
                                print(f'\tWaiting_cost[{scenery}] -> {waiting_costs[scenery]}')
                                copied_slot_apps[scenery][slot+1].insert(idx_insertion, patient)
                                idx_insertion += 1
                                print(f'\tIdx_insertion = {idx_insertion}')
                        elif set.loc[patient_row_index, 'NoShow'].any() == False:
                            slot_ocupado = True
                            idle_costs[scenery] -= 1
                            print(f'\tIdle_cost[{scenery}] -> {idle_costs[scenery]}')
                    slot += 1
    
        average_idle_cost = sum(list(idle_costs.values())) / len(slot_apps)
        print(f'\nLa media de los Costes de Inactividad en los {NUM_SETTINGS} escenarios distintos \
    para las consultas de los {NUM_DOCTORS} doctores distintos es de {average_idle_cost:.2f}.')
    
        average_waiting_cost = sum(list(waiting_costs.values())) / len(slot_apps)
        print(f'La media de los Costes de Espera de los Pacientes en los {NUM_SETTINGS} escenarios distintos \
    para las consultas de los {NUM_DOCTORS} doctores distintos es de {average_waiting_cost:.2f}.')
    
        average_overtime_cost = sum(list(overtime_costs.values())) / len(slot_apps)
        print(f'La media de los Costes de Tiempo Extra en los {NUM_SETTINGS} escenarios distintos \
    para las consultas de los {NUM_DOCTORS} doctores distintos es de {average_overtime_cost:.2f}.')
        
        costs = {'idle_costs': idle_costs,
                'waiting_costs': waiting_costs,
                'overtime_costs': overtime_costs}
        average_costs = {'average_idle_cost': average_idle_cost,
                        'average_waiting_cost': average_waiting_cost,
                        'average_overtime_cost': average_overtime_cost}
        
        return costs, average_costs

    # 3 Junio 2016:
    print('\033[1m03 de junio:\033[0m')
    print('-----------')
    print('Set Total con todas las Citas Médicas:')
    costs_total_June03_R11, average_costs_total_June03_R11 = calculo_costes(
        set = total_set_June03,
        groups = total_groups_June03_R11,
        slot_apps = slot_apps_total_June03_R11
    )
    print('------------------------------------------------------------------\
    -------------------------------------------------------------------------\n')

    # 3 Junio 2016
    print('\033[1m03 de junio:\033[0m')
    print('-----------')
    print('Set Total con todas las Citas Médicas:')
    costs_total_June03_R1231, average_costs_total_June03_R1231 = calculo_costes(
        set = total_set_June03,
        groups = total_groups_June03_R1231,
        slot_apps = slot_apps_total_June03_R1231
    )
    print('------------------------------------------------------------------\
    -------------------------------------------------------------------------\n')


In [15]:
print('\033[1mHIPÓTESIS: Regla de Asignación de Cita\n \
\t\t 1 Slot : 1 Patient\033[0m\n')

# 1 Junio 2016
print('\033[1m01 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June01_R11, average_costs_total_June01_R11 = calculo_costes(
    set = total_set_June01,
    groups = total_groups_June01_R11,
    slot_apps = slot_apps_total_June01_R11
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 2 Junio 2016
print('\033[1m02 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June02_R11, average_costs_total_June02_R11 = calculo_costes(
    set = total_set_June02,
    groups = total_groups_June02_R11,
    slot_apps = slot_apps_total_June02_R11
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 3 Junio 2016:
print('\033[1m03 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June03_R11, average_costs_total_June03_R11 = calculo_costes(
    set = total_set_June03,
    groups = total_groups_June03_R11,
    slot_apps = slot_apps_total_June03_R11
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 6 Junio 2016
print('\033[1m06 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June06_R11, average_costs_total_June06_R11 = calculo_costes(
    set = total_set_June06,
    groups = total_groups_June06_R11,
    slot_apps = slot_apps_total_June06_R11
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 7 Junio 2016
print('\033[1m07 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June07_R11, average_costs_total_June07_R11 = calculo_costes(
    set = total_set_June07,
    groups = total_groups_June07_R11,
    slot_apps = slot_apps_total_June07_R11
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 8 Junio 2016
print('\033[1m08 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June08_R11, average_costs_total_June08_R11 = calculo_costes(
    set = total_set_June08,
    groups = total_groups_June08_R11,
    slot_apps = slot_apps_total_June08_R11
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')


HIPÓTESIS: Regla de Asignación de Cita
 		 1 Slot : 1 Patient

01 de junio:
-----------
Set Total con todas las Citas Médicas:
Número de consultas: 1270  =  10 settings diferentes * 127 doctores pasando consulta.
La media de los Costes de Inactividad en los 10 escenarios distintos para las consultas de los 127 doctores distintos es de 4.50.
La media de los Costes de Espera de los Pacientes en los 10 escenarios distintos para las consultas de los 127 doctores distintos es de 0.00.
La media de los Costes de Tiempo Extra en los 10 escenarios distintos para las consultas de los 127 doctores distintos es de 0.00.
-------------------------------------------------------------------------------------------------------------------------------------------

02 de junio:
-----------
Set Total con todas las Citas Médicas:
Número de consultas: 1350  =  10 settings diferentes * 135 doctores pasando consulta.
La media de los Costes de Inactividad en los 10 escenarios distintos para las consultas de lo

In [16]:
# Crear la Tabla RESUMEN con los costes medios para esta Hipótesis R11
data_R11 = {
    'Fecha': [
        '2016-06-01', '2016-06-02', '2016-06-03',
        '2016-06-06', '2016-06-07', '2016-06-08'
    ],
    'Settings': [
        len(total_groups_June01_R11),
        len(total_groups_June02_R11),
        len(total_groups_June03_R11),
        len(total_groups_June06_R11),
        len(total_groups_June07_R11),
        len(total_groups_June08_R11)
    ],
    'Consultas': [
        len(total_groups_June01_R11['setting 1']),
        len(total_groups_June02_R11['setting 1']),
        len(total_groups_June03_R11['setting 1']),
        len(total_groups_June06_R11['setting 1']),
        len(total_groups_June07_R11['setting 1']),
        len(total_groups_June08_R11['setting 1']),
    ],
    'Idle Cost': [
        average_costs_total_June01_R11['average_idle_cost'],
        average_costs_total_June02_R11['average_idle_cost'],
        average_costs_total_June03_R11['average_idle_cost'],
        average_costs_total_June06_R11['average_idle_cost'],
        average_costs_total_June07_R11['average_idle_cost'],
        average_costs_total_June08_R11['average_idle_cost']
    ],
    'Waiting Cost': [
        average_costs_total_June01_R11['average_waiting_cost'],
        average_costs_total_June02_R11['average_waiting_cost'],
        average_costs_total_June03_R11['average_waiting_cost'],
        average_costs_total_June06_R11['average_waiting_cost'],
        average_costs_total_June07_R11['average_waiting_cost'],
        average_costs_total_June08_R11['average_waiting_cost']
    ],
    'Overbooking Cost': [
        average_costs_total_June01_R11['average_overtime_cost'],
        average_costs_total_June02_R11['average_overtime_cost'],
        average_costs_total_June03_R11['average_overtime_cost'],
        average_costs_total_June06_R11['average_overtime_cost'],
        average_costs_total_June07_R11['average_overtime_cost'],
        average_costs_total_June08_R11['average_overtime_cost']
    ]
}

tabla_resumen_R11 = pd.DataFrame(data_R11)

# Mostrar el DataFrame
tabla_resumen_R11

,Fecha,Settings,Consultas,Idle Cost,Waiting Cost,Overbooking Cost
0,2016-06-01,10,127,4.502362,0.0,0.0
1,2016-06-02,10,135,4.388148,0.0,0.0
2,2016-06-03,10,131,4.704580,0.0,0.0
3,2016-06-06,10,158,4.492405,0.0,0.0
4,2016-06-07,10,161,4.227950,0.0,0.0
5,2016-06-08,10,171,4.115205,0.0,0.0


In [17]:
# Exportar el DataFrame a Excel

# Crear un nuevo workbook
wb = Workbook()
ws = wb.active

# Añadir el título
ws.merge_cells('A1:F1')
ws['A1'] = 'Costes Promedio HipótesisR11'
ws['A1'].alignment = Alignment(horizontal = 'center')

# Añadir el DataFrame a la hoja de cálculo
for r_idx, row in enumerate(dataframe_to_rows(tabla_resumen_R11,
                                                index = False,
                                                header = True), 2):
    for c_idx, value in enumerate(row, 1):
        cell = ws.cell(row = r_idx, column = c_idx, value = value)
        cell.alignment = Alignment(horizontal='center')
        if isinstance(value, float) and c_idx > 2:
            cell.number_format = '0.0000'

# Ajustar el ancho de las columnas
for col in ws.iter_cols(min_row = 2,
                        max_row = ws.max_row,
                        min_col = 1,
                        max_col = ws.max_column):
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        if cell.coordinate in ws.merged_cells:  # no considerar celdas fusionadas
            continue
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2)
    ws.column_dimensions[column].width = adjusted_width

# Guardar el archivo
wb.save('Costes_promedio_R11.xlsx')

# Mostrar mensaje de confirmación
print("El DataFrame ha sido exportado exitosamente a 'Costes_promedio_R11.xlsx'")

El DataFrame ha sido exportado exitosamente a 'Costes_promedio_R11.xlsx'


## 2.2 Regla de Asignación de Cita
### **1 Slot : 2 Patients -> 3 Slots : 1 Patient -> 1 Slot : 2 Patients ...**

24 Slots - 30 pacientes citados

### 2.2.1 Asignaciones de cita

In [18]:
NUM_SLOTS_PER_DAY = 24

# 01 Junio 2016:
# 10 settings x 102 consultas = 1020 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June01_R1231 = {}
for i, setting in enumerate(total_groups_June01_R1231.keys()):
    for j, group in enumerate(total_groups_June01_R1231[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June01_R1231[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 5, 9, 13, 17, 21]:
                slot_apps_total_June01_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June01_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

# 02 Junio 2016:
# 10 settings x 108 consultas = 1080 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June02_R1231 = {}
for i, setting in enumerate(total_groups_June02_R1231.keys()):
    for j, group in enumerate(total_groups_June02_R1231[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June02_R1231[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 5, 9, 13, 17, 21]:
                slot_apps_total_June02_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June02_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

# 03 Junio 2016:
# 10 settings x 105 consultas = 1050 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June03_R1231 = {}
for i, setting in enumerate(total_groups_June03_R1231.keys()):
    for j, group in enumerate(total_groups_June03_R1231[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June03_R1231[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 5, 9, 13, 17, 21]:
                slot_apps_total_June03_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June03_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

# 06 Junio 2016:
# 10 settings x 126 consultas = 1260 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June06_R1231 = {}
for i, setting in enumerate(total_groups_June06_R1231.keys()):
    for j, group in enumerate(total_groups_June06_R1231[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June06_R1231[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 5, 9, 13, 17, 21]:
                slot_apps_total_June06_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June06_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

# 07 Junio 2016:
# 10 settings x 129 consultas = 1290 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June07_R1231 = {}
for i, setting in enumerate(total_groups_June07_R1231.keys()):
    for j, group in enumerate(total_groups_June07_R1231[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June07_R1231[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 5, 9, 13, 17, 21]:
                slot_apps_total_June07_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June07_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

# 08 Junio 2016:
# 10 settings x 137 consultas = 1370 diccionarios de 24 diccionarios (número slots) de listas (pacientes asignados)
slot_apps_total_June08_R1231 = {}
for i, setting in enumerate(total_groups_June08_R1231.keys()):
    for j, group in enumerate(total_groups_June08_R1231[setting]):
        group_name = f'slot_apps_{i+1}_{j+1}'
        slot_apps_total_June08_R1231[group_name] = {}
        idx_slot = 0
        for slot in range(NUM_SLOTS_PER_DAY):
            if slot+1 in [1, 5, 9, 13, 17, 21]:
                slot_apps_total_June08_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId'],
                                                                group.iloc[idx_slot+1].loc['PatientId']]
                idx_slot += 2
            else:
                slot_apps_total_June08_R1231[group_name][slot+1] = [group.iloc[idx_slot].loc['PatientId']]
                idx_slot += 1

### 2.2.2 Cálculo de Costes: Idle Time, Waiting Time y Over Time

In [19]:
print('\033[1mHIPÓTESIS: Regla de Asignación de Cita\n \
\t\t 1 Slot : 2 Patients -> 3 Slots : 1 Patient -> 1 Slot : 2 Patients ...\033[0m\n')

# 1 Junio 2016
print('\033[1m01 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June01_R1231, average_costs_total_June01_R1231 = calculo_costes(
    set = total_set_June01,
    groups = total_groups_June01_R1231,
    slot_apps = slot_apps_total_June01_R1231
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 2 Junio 2016
print('\033[1m02 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June02_R1231, average_costs_total_June02_R1231 = calculo_costes(
    set = total_set_June02,
    groups = total_groups_June02_R1231,
    slot_apps = slot_apps_total_June02_R1231
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 3 Junio 2016
print('\033[1m03 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June03_R1231, average_costs_total_June03_R1231 = calculo_costes(
    set = total_set_June03,
    groups = total_groups_June03_R1231,
    slot_apps = slot_apps_total_June03_R1231
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 6 Junio 2016
print('\033[1m06 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June06_R1231, average_costs_total_June06_R1231 = calculo_costes(
    set = total_set_June06,
    groups = total_groups_June06_R1231,
    slot_apps = slot_apps_total_June06_R1231
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 7 Junio 2016
print('\033[1m07 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June07_R1231, average_costs_total_June07_R1231 = calculo_costes(
    set = total_set_June07,
    groups = total_groups_June07_R1231,
    slot_apps = slot_apps_total_June07_R1231
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

# 8 Junio 2016
print('\033[1m08 de junio:\033[0m')
print('-----------')
print('Set Total con todas las Citas Médicas:')
costs_total_June08_R1231, average_costs_total_June08_R1231 = calculo_costes(
    set = total_set_June08,
    groups = total_groups_June08_R1231,
    slot_apps = slot_apps_total_June08_R1231
)
print('------------------------------------------------------------------\
-------------------------------------------------------------------------\n')

HIPÓTESIS: Regla de Asignación de Cita
 		 1 Slot : 2 Patients -> 3 Slots : 1 Patient -> 1 Slot : 2 Patients ...

01 de junio:
-----------
Set Total con todas las Citas Médicas:
Número de consultas: 1020  =  10 settings diferentes * 102 doctores pasando consulta.
La media de los Costes de Inactividad en los 10 escenarios distintos para las consultas de los 102 doctores distintos es de 2.00.
La media de los Costes de Espera de los Pacientes en los 10 escenarios distintos para las consultas de los 102 doctores distintos es de 21.16.
La media de los Costes de Tiempo Extra en los 10 escenarios distintos para las consultas de los 102 doctores distintos es de 2.37.
-------------------------------------------------------------------------------------------------------------------------------------------

02 de junio:
-----------
Set Total con todas las Citas Médicas:
Número de consultas: 1080  =  10 settings diferentes * 108 doctores pasando consulta.
La media de los Costes de Inactividad en 

In [20]:
# Crear la Tabla RESUMEN con los costes medios para esta Hipótesis R1231
data_R1231 = {
    'Fecha': [
        '2016-06-01', '2016-06-02', '2016-06-03',
        '2016-06-06', '2016-06-07', '2016-06-08'
    ],
    'Settings': [
        len(total_groups_June01_R1231),
        len(total_groups_June02_R1231),
        len(total_groups_June03_R1231),
        len(total_groups_June06_R1231),
        len(total_groups_June07_R1231),
        len(total_groups_June08_R1231)
    ],
    'Consultas': [
        len(total_groups_June01_R1231['setting 1']),
        len(total_groups_June02_R1231['setting 1']),
        len(total_groups_June03_R1231['setting 1']),
        len(total_groups_June06_R1231['setting 1']),
        len(total_groups_June07_R1231['setting 1']),
        len(total_groups_June08_R1231['setting 1']),
    ],
    'Idle Cost': [
        average_costs_total_June01_R1231['average_idle_cost'],
        average_costs_total_June02_R1231['average_idle_cost'],
        average_costs_total_June03_R1231['average_idle_cost'],
        average_costs_total_June06_R1231['average_idle_cost'],
        average_costs_total_June07_R1231['average_idle_cost'],
        average_costs_total_June08_R1231['average_idle_cost']
    ],
    'Waiting Cost': [
        average_costs_total_June01_R1231['average_waiting_cost'],
        average_costs_total_June02_R1231['average_waiting_cost'],
        average_costs_total_June03_R1231['average_waiting_cost'],
        average_costs_total_June06_R1231['average_waiting_cost'],
        average_costs_total_June07_R1231['average_waiting_cost'],
        average_costs_total_June08_R1231['average_waiting_cost']
    ],
    'Overbooking Cost': [
        average_costs_total_June01_R1231['average_overtime_cost'],
        average_costs_total_June02_R1231['average_overtime_cost'],
        average_costs_total_June03_R1231['average_overtime_cost'],
        average_costs_total_June06_R1231['average_overtime_cost'],
        average_costs_total_June07_R1231['average_overtime_cost'],
        average_costs_total_June08_R1231['average_overtime_cost']
    ]
}

tabla_resumen_R1231 = pd.DataFrame(data_R1231)

# Mostrar el DataFrame
tabla_resumen_R1231


,Fecha,Settings,Consultas,Idle Cost,Waiting Cost,Overbooking Cost
0,2016-06-01,10,102,1.995098,21.155882,2.369608
1,2016-06-02,10,108,1.916667,21.423148,2.431481
2,2016-06-03,10,105,2.045714,19.973333,2.163810
3,2016-06-06,10,126,2.061111,21.428571,2.445238
4,2016-06-07,10,129,1.727132,22.693023,2.436434
5,2016-06-08,10,137,1.568613,22.103650,2.427007


In [21]:
# Exportar el DataFrame a Excel

# Crear un nuevo workbook
wb = Workbook()
ws = wb.active

# Añadir el título
ws.merge_cells('A1:F1')
ws['A1'] = 'Costes Promedio HipótesisR1231'
ws['A1'].alignment = Alignment(horizontal='center')

# Añadir el DataFrame a la hoja de cálculo
for r_idx, row in enumerate(dataframe_to_rows(tabla_resumen_R1231,
                                                index = False,
                                                header = True), 2):
    for c_idx, value in enumerate(row, 1):
        cell = ws.cell(row = r_idx, column = c_idx, value = value)
        cell.alignment = Alignment(horizontal='center')
        if isinstance(value, float) and c_idx > 2:
            cell.number_format = '0.0000'

# Ajustar el ancho de las columnas
for col in ws.iter_cols(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        if cell.coordinate in ws.merged_cells:  # no considerar celdas fusionadas
            continue
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2)
    ws.column_dimensions[column].width = adjusted_width

# Guardar el archivo
wb.save('Costes_promedio_R1231.xlsx')

# Mostrar mensaje de confirmación
print("El DataFrame ha sido exportado exitosamente a 'Costes_promedio_R1231.xlsx'")

El DataFrame ha sido exportado exitosamente a 'Costes_promedio_R1231.xlsx'
